In [24]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()  # for plot styling
import numpy as np
import pandas as pd
from pandas_datareader import data as pdr
from yahoo_fin import stock_info as si
from pandas import ExcelWriter
import yfinance as yf
import pandas as pd
import requests
import datetime
import time
import warnings
import ta
warnings.filterwarnings('ignore')

In [25]:
stocklist = si.tickers_sp500()
start_date = datetime.datetime.now() - datetime.timedelta(days=(365*3) + 30)
end_date = datetime.date.today()

In [26]:
def get_data(ticker,start_date,end_date):
    data = pdr.get_data_yahoo(ticker, start=start_date, end=end_date)
    data['Ticker'] = ticker
    return data


def technical_analysis(sec):
    '''
    Contains all the indicators
    sec: security dataframe - should contain Close, Open, Low, High, and Volume of the desired security
    returns all the technical indicators
    
    '''
    #Momentum Indicators
    stoc_oscc_signal = ta.momentum.stoch_signal(high = sec.High, low = sec.Low,close = sec.Close, n=14, d_n=3, fillna=False)
    tsi = ta.momentum.tsi(close = sec.Close, r=25, s=13, fillna=False)
    awesome_oscillator = ta.momentum.AwesomeOscillatorIndicator(high = sec.High, low = sec.Low, s = 5, len = 34, fillna = False).ao()
    kama_indicator = ta.momentum.KAMAIndicator(close = sec.Close, n = 10, pow1 = 2, pow2 = 30, fillna = False).kama()
    roc_indicator = ta.momentum.ROCIndicator(close = sec.Close, n = 12, fillna = False).roc()
    rsi14 = ta.momentum.rsi(close = sec.Close, n=14, fillna=False)
    ultimate_osc = ta.momentum.UltimateOscillator(high = sec.High, low = sec.Low,close =  sec.Close, s = 7, m = 14, len = 28, ws = 4.0, wm = 2.0, wl = 1.0, fillna = False).uo()
    williamsR = ta.momentum.WilliamsRIndicator(high = sec.High, low = sec.Low,close =  sec.Close, lbp = 14, fillna = False).wr()
    
    #Volume Indicators
    adi = ta.volume.AccDistIndexIndicator(high = sec.High, low = sec.Low, close = sec.Close, volume = sec.Volume, fillna = False).acc_dist_index()
    chaikin_money_flow = ta.volume.ChaikinMoneyFlowIndicator(high = sec.High, low = sec.Low, close = sec.Close, volume = sec.Volume, n= 20, fillna = False).chaikin_money_flow()
    #ease_of_movement = ta.volume.EaseOfMovementIndicator(high = sec.High, low = sec.Low, volume = sec.Volume, n= 14, fillna = False).ease_of_movement()
    force_index_indicator = ta.volume.ForceIndexIndicator(close = sec.Close, volume = sec.Volume, n = 13, fillna = False).force_index()
    mfi = ta.volume.MFIIndicator(high = sec.High, low = sec.Low, close = sec.Close, volume= sec.Volume, n = 14, fillna = False).money_flow_index()
    nvi = ta.volume.NegativeVolumeIndexIndicator(close = sec.Close, volume = sec.Volume, fillna= False).negative_volume_index()
    obv = ta.volume.OnBalanceVolumeIndicator(close = sec.Close, volume = sec.Volume, fillna = False).on_balance_volume()
    priceTrend = ta.volume.VolumePriceTrendIndicator(close = sec.Close, volume = sec.Volume, fillna = False).volume_price_trend()
    vwap = ta.volume.VolumeWeightedAveragePrice(high = sec.High, low = sec.Low, close = sec.Close, volume = sec.Volume, n= 14, fillna= False).volume_weighted_average_price()
    
    #Volatility Indicators
    atr = ta.volatility.AverageTrueRange(high = sec.High, low = sec.Low, close = sec.Close, n= 14, fillna= False).average_true_range()
    
    #Trend Indicators
    adx = ta.trend.ADXIndicator(high = sec.High, low = sec.Low, close = sec.Close, n= 14, fillna = False).adx()
    aroon = ta.trend.AroonIndicator(close = sec.Close, n = 25, fillna = False).aroon_indicator()
    cci = ta.trend.CCIIndicator(high = sec.High, low = sec.Low, close = sec.Close, n = 20, c= 0.015, fillna= False).cci()
    dpo = ta.trend.DPOIndicator(close = sec.Close, n = 20, fillna = False).dpo()
    ema = ta.trend.EMAIndicator(close = sec.Close, n = 14, fillna = False).ema_indicator()
    kst = ta.trend.KSTIndicator(close= sec.Close, r1 = 10, r2 = 15, r3= 20, r4 = 30, n1= 10, n2= 10, n3 = 10, n4 = 15, nsig = 9, fillna= False).kst()
    macd = ta.trend.MACD(close = sec.Close, n_slow = 26, n_fast = 12, n_sign= 9, fillna = False).macd()
    
    tech = pd.DataFrame({'stoc_oscc_signal':stoc_oscc_signal,'tsi':tsi,'awesome_oscillator':awesome_oscillator,'kama_indicator':kama_indicator,
    'roc_indicator':roc_indicator,'rsi14':rsi14,'ultimate_osc':ultimate_osc,'williamsR':williamsR,
    'adi':adi,"chaikin_money_flow":chaikin_money_flow,'force_index_indicator':force_index_indicator,
    'mfi':mfi,'nvi':nvi,'obv':obv,'priceTrend':priceTrend,'vwap':vwap,'atr':atr,
    'adx':adx,'aroon':aroon,'cci':cci,'dpo':dpo,'ema':ema, 'kst':kst,'macd':macd})
    
    return tech


def merge_returns_and_technicals(price_df,technicals_df):
    ticker = price_df['Ticker'][0]
    price_df.drop(columns = ['Ticker'],inplace = True)
    returns = price_df.pct_change(1).rename(columns = {'Adj Close': 'Returns'})
    merged_df = technicals_df.join(returns['Returns'].shift(1)) #Lagged Returns
    merged_df['Ticker'] = ticker
    return merged_df


def master_data(security_list,start_date,end_date):
    final_df = pd.DataFrame()
    count = 0
    for security in security_list:
        try:
            sec = get_data(security,start_date = start_date,end_date = end_date)
            tech = technical_analysis(sec)
            merged = merge_returns_and_technicals(sec,tech)
            final_df = pd.concat([final_df,merged],axis = 0).dropna()
        except:
            pass
        count+=1
        print(count)
    return final_df

In [27]:
master = master_data(stocklist,start_date=start_date,end_date=end_date)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [28]:
master.head()

,stoc_oscc_signal,tsi,awesome_oscillator,kama_indicator,roc_indicator,rsi14,ultimate_osc,williamsR,adi,chaikin_money_flow,...,atr,adx,aroon,cci,dpo,ema,kst,macd,Returns,Ticker
Date,,,,,,,,,,,,,,,,,,,,,
2017-09-13,88.474027,40.772693,3.492823,65.121088,4.038157,67.034183,59.478050,-24.078521,1.120261e+06,0.183521,...,0.875120,32.424460,88.0,105.144495,-0.245001,64.463531,37.203606,1.241264,0.000454,A
2017-09-14,83.102875,40.387983,3.457265,65.176526,3.498269,68.294794,60.522064,-19.733276,1.808602e+06,0.090565,...,0.863326,32.449100,88.0,89.665386,-0.391998,64.625727,42.745578,1.228560,-0.010584,A
2017-09-15,80.951628,40.460908,3.406118,65.231197,3.890372,69.748247,58.790645,-13.333319,2.573936e+06,0.222675,...,0.838088,32.695931,84.0,98.679634,-0.236999,64.803630,47.844852,1.226943,0.003667,A
2017-09-18,85.644502,40.687682,3.270236,65.338366,3.411581,70.375074,56.758512,-9.999899,2.990474e+06,0.214929,...,0.821082,33.070699,76.0,99.099524,0.341501,64.973813,53.484241,1.221266,0.004263,A
2017-09-19,89.172163,41.002693,3.102618,65.462352,2.240416,70.862676,58.533635,-9.150294,2.687848e+06,0.180465,...,0.794576,33.548684,96.0,103.289087,-0.220003,65.133304,58.929031,1.210080,0.001819,A


In [29]:
master.columns

Index(['stoc_oscc_signal', 'tsi', 'awesome_oscillator', 'kama_indicator',
       'roc_indicator', 'rsi14', 'ultimate_osc', 'williamsR', 'adi',
       'chaikin_money_flow', 'force_index_indicator', 'mfi', 'nvi', 'obv',
       'priceTrend', 'vwap', 'atr', 'adx', 'aroon', 'cci', 'dpo', 'ema', 'kst',
       'macd', 'Returns', 'Ticker'],
      dtype='object')

In [30]:
master = master[['Ticker','Returns','stoc_oscc_signal', 'tsi', 'awesome_oscillator', 'kama_indicator',
       'roc_indicator', 'rsi14', 'ultimate_osc', 'williamsR', 'adi',
       'chaikin_money_flow', 'force_index_indicator',
       'mfi', 'nvi', 'obv', 'priceTrend', 'vwap', 'atr', 'adx', 'aroon', 'cci',
       'dpo', 'ema', 'kst', 'macd']]

In [32]:
master.describe().T

,count,mean,std,min,25%,50%,75%,max
Returns,365595.0,5.338013e-04,2.391108e-02,-5.655036e-01,-8.267049e-03,1.041042e-03,9.885499e-03,6.797965e-01
stoc_oscc_signal,365595.0,5.584629e+01,2.804942e+01,2.091966e-01,3.081464e+01,5.947956e+01,8.164457e+01,9.979288e+01
tsi,365595.0,4.494735e+00,2.131022e+01,-7.353208e+01,-9.837593e+00,4.873056e+00,1.940623e+01,7.984511e+01
awesome_oscillator,365595.0,8.327404e-01,1.439613e+01,-1.184371e+03,-1.502764e+00,5.942946e-01,3.022896e+00,4.777834e+02
kama_indicator,365595.0,1.256985e+02,2.121524e+02,3.699683e+00,4.819494e+01,8.200788e+01,1.385739e+02,4.062966e+03
roc_indicator,365595.0,5.236170e-01,7.696589e+00,-8.260188e+01,-2.777177e+00,8.996526e-01,4.201565e+00,1.109453e+02
rsi14,365595.0,5.242561e+01,1.241061e+01,8.255315e+00,4.411749e+01,5.277657e+01,6.118823e+01,9.295193e+01
ultimate_osc,365595.0,5.155274e+01,1.010248e+01,1.239535e+01,4.468880e+01,5.168728e+01,5.854345e+01,8.781656e+01
williamsR,365595.0,-4.417296e+01,3.009473e+01,-1.000000e+02,-7.052982e+01,-4.042859e+01,-1.675391e+01,-0.000000e+00
adi,365595.0,6.129536e+06,2.084881e+08,-5.995569e+09,-8.449672e+06,9.289070e+06,3.369015e+07,1.675219e+09


In [11]:
y = master['Returns']

In [12]:
master.columns

Index(['Ticker', 'Returns', 'stoc_oscc_signal', 'tsi', 'awesome_oscillator',
       'kama_indicator', 'roc_indicator', 'rsi14', 'ultimate_osc', 'williamsR',
       'adi', 'chaikin_money_flow', 'ease_of_movement',
       'force_index_indicator', 'mfi', 'nvi', 'obv', 'priceTrend', 'vwap',
       'atr', 'adx', 'aroon', 'cci', 'dpo', 'ema', 'kst', 'macd'],
      dtype='object')

In [13]:
X  = master[['stoc_oscc_signal', 'tsi', 'awesome_oscillator',
       'kama_indicator', 'roc_indicator', 'rsi14', 'ultimate_osc', 'williamsR',
       'adi', 'chaikin_money_flow', 'ease_of_movement',
       'force_index_indicator', 'mfi', 'nvi', 'obv', 'priceTrend', 'vwap',
       'atr', 'adx', 'aroon', 'cci', 'dpo', 'ema', 'kst', 'macd']]